# Python 08 — Introdução prática a Machine Learning com Python (Iris, scikit-learn e TensorFlow)

Este notebook é uma aula completa e prática cobrindo conceitos básicos de Machine Learning (ML) com Python, usando dois ecossistemas populares: **scikit-learn** (clássico/estatístico) e **TensorFlow/Keras** (redes neurais). Trabalharemos com o conjunto de dados clássico **Iris**.

Objetivos:
- Entender os conceitos fundamentais de ML supervisionado (features, rótulos, treino/teste, validação, métricas).
- Explorar o dataset Iris e realizar EDA (Análise Exploratória de Dados) básica.
- Treinar modelos com scikit-learn (Logistic Regression, SVC) usando boas práticas (Pipeline, padronização, validação cruzada, Grid Search).
- Salvar e carregar modelos treinados.
- Treinar uma rede neural simples com TensorFlow/Keras e comparar resultados.

Pré-requisitos (recomendado): Python 3.9+ e as bibliotecas `numpy`, `pandas`, `matplotlib`, `seaborn`, `scikit-learn`, `tensorflow`.

## 0) Instalação (opcional)
Se estiver rodando localmente e ainda não tiver as dependências, execute a célula abaixo. Ela usa o interpretador atual do Python para instalar os pacotes necessários.

Observação: a instalação do TensorFlow pode demorar e requer ambiente compatível. No Colab, geralmente não é necessário.

In [ ]:
# Instalação opcional de dependências (compatível com Colab e Jupyter)
# - Detecta Colab e instala pacotes necessários usando o mesmo interpretador do kernel.
# - Se já tiver tudo instalado, pode pular.

from __future__ import annotations
import sys, subprocess

try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:  # noqa: BLE001
    IN_COLAB = False

PKGS = [
    'numpy', 'pandas', 'matplotlib', 'seaborn', 'scikit-learn', 'joblib', 'tensorflow'
]

def pip_install(pkgs: list[str]) -> None:
    cmd = [sys.executable, '-m', 'pip', 'install', '-q', '-U', *pkgs]
    print('Instalando:', ' '.join(pkgs))
    subprocess.check_call(cmd)

if IN_COLAB:
    print('Ambiente Colab detectado. As dependências já devem estar disponíveis.')
    # pip_install(PKGS) # Descomente se precisar forçar a atualização
else:
    print('Instalação opcional — pule se o ambiente já tiver as dependências.')


## 1) Imports, Configurações e Verificação de Versões

- Importar bibliotecas e módulos.
- Definir sementes de aleatoriedade para reprodutibilidade.
- Verificar versões para diagnosticar problemas de ambiente.

In [ ]:
import sys
import os
import random
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import sklearn

from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Configs de plot
sns.set(theme="whitegrid", context="notebook")
plt.rcParams["figure.figsize"] = (8, 5)

# Semente para reprodutibilidade
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

# Verificação de versões
print("Python:", sys.version.split()[0])
print("Numpy:", np.__version__)
print("Pandas:", pd.__version__)
print("scikit-learn:", sklearn.__version__)
print("TensorFlow:", tf.__version__)

## 2) Conceitos básicos de ML supervisionado

- **Problema**: Classificar a espécie de uma flor (Iris) a partir de suas medidas (features).
- **Features (X)**: Comprimento/largura de sépalas e pétalas (4 variáveis numéricas).
- **Rótulo/Target (y)**: Espécie da flor (3 classes: setosa, versicolor, virginica).
- **Fluxo padrão**: Dividir dados em treino/teste, treinar o modelo nos dados de treino, validar e, finalmente, avaliar no conjunto de teste.
- **Métricas comuns**: Acurácia, precisão, revocação, F1-score.
- **Boas práticas**: Padronização dos dados, validação cruzada, manter o conjunto de teste intocado até a avaliação final.

## 3) Carregando o dataset Iris e Análise Exploratória Rápida (EDA)

Usaremos `sklearn.datasets.load_iris` e o converteremos para um `pandas.DataFrame` para facilitar a inspeção.

In [ ]:
iris = datasets.load_iris(as_frame=True)
df: pd.DataFrame = iris.frame.copy()
df.head()

In [ ]:
print("Colunas:", list(df.columns))
print("Classes (target_names):", list(iris.target_names))
print("Descrição resumida:\n", '\n'.join(iris.DESCR.split('\n')[0:5]))

# Mapear target numérico -> rótulo textual para visualizações
df["species"] = df["target"].map(dict(enumerate(iris.target_names)))
df.sample(5, random_state=SEED)

### 3.1) Estatísticas e Balanceamento de Classes

Vamos verificar as estatísticas descritivas e se as classes estão balanceadas.

In [ ]:
# Estatísticas descritivas
display(df.describe(include="all").T)

# Distribuição de classes
plt.title("Distribuição das Classes no Dataset Iris")
ax = sns.countplot(data=df, x="species", order=sorted(df.species.unique()))
ax.bar_label(ax.containers[0])
plt.xlabel("Espécie")
plt.ylabel("Quantidade")
plt.show()

### 3.2) Relações entre variáveis

Um `pairplot` ajuda a visualizar a separabilidade entre as classes (pode demorar um pouco).

In [ ]:
sns.pairplot(df, vars=iris.feature_names, hue="species", diag_kind="kde")
plt.suptitle("Pairplot do Dataset Iris", y=1.02)
plt.show()

## 4) Treino/Teste, Pipeline e Primeiro Modelo (scikit-learn)

- Separação treino/teste com estratificação para manter a proporção das classes.
- Uso de um `Pipeline` simples com Regressão Logística.
- Métricas: acurácia, relatório de classificação e matriz de confusão.

In [ ]:
# Features (X) e alvo (y)
X = df[iris.feature_names].to_numpy(dtype=np.float32)
y = df["target"].to_numpy(dtype=np.int64)

# Divisão em treino e teste (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de X_test: {X_test.shape}")

# Criação e treino do pipeline com Regressão Logística
pipe_lr = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=1000, random_state=SEED)),
])
pipe_lr.fit(X_train, y_train)

# Previsão e avaliação
pred_lr = pipe_lr.predict(X_test)
acc_lr = accuracy_score(y_test, pred_lr)
print(f"Acurácia (LogisticRegression): {acc_lr:.4f}\n")

print("Relatório de classificação:\n", classification_report(y_test, pred_lr, target_names=iris.target_names))

# Matriz de confusão
cm = confusion_matrix(y_test, pred_lr)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title("Matriz de confusão — LogisticRegression")
plt.xlabel("Predito")
plt.ylabel("Verdadeiro")
plt.show()

## 5) Validação Cruzada e Seleção de Hiperparâmetros (Grid Search com SVC)

Vamos usar um `Pipeline` com `StandardScaler` + `SVC` e procurar os melhores hiperparâmetros (`C` e `gamma`) usando validação cruzada (k-fold).

In [ ]:
pipe_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("svc", SVC(random_state=SEED))
])

param_grid = {
    "svc__C": [0.1, 1, 10, 100],
    "svc__gamma": ["scale", "auto", 0.01, 0.1],
    "svc__kernel": ["rbf", "linear"]
}

grid = GridSearchCV(
    estimator=pipe_svc,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1, # Usar todos os processadores
    verbose=0
)

grid.fit(X_train, y_train)

print("Melhores parâmetros:", grid.best_params_)
print(f"Melhor acurácia (CV): {grid.best_score_:.4f}\n")

# Avaliação do melhor modelo encontrado no conjunto de teste
best_model = grid.best_estimator_
pred_svc = best_model.predict(X_test)
acc_svc = accuracy_score(y_test, pred_svc)

print(f"Acurácia em teste (SVC): {acc_svc:.4f}\n")
print("Relatório de classificação (SVC):\n", classification_report(y_test, pred_svc, target_names=iris.target_names))

# Matriz de confusão
cm2 = confusion_matrix(y_test, pred_svc)
sns.heatmap(cm2, annot=True, fmt="d", cmap="Greens",
            xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title("Matriz de confusão — SVC (melhor modelo)")
plt.xlabel("Predito")
plt.ylabel("Verdadeiro")
plt.show()

### 5.1) Validação Cruzada Rápida

Exemplo de como verificar a performance média de um estimador via `cross_val_score` em todo o dataset.

In [ ]:
cv_scores = cross_val_score(pipe_lr, X, y, cv=5, scoring="accuracy")
print("Scores da Validação Cruzada (5-fold):", np.round(cv_scores, 4))
print(f"Média: {cv_scores.mean():.4f} | Desvio Padrão: {cv_scores.std():.4f}")

## 6) Persistência do Modelo (Salvar/Carregar)

Salvar modelos é essencial para deploy e reuso. Usaremos `joblib`, que é eficiente para objetos NumPy.

In [ ]:
# Salvar o melhor modelo (pipeline completo)
out_dir = Path("./models")
out_dir.mkdir(exist_ok=True)
model_path = out_dir / "iris_svc_pipeline.joblib"

joblib.dump(best_model, model_path)
print(f"Modelo salvo em: {model_path.resolve()}\n")

# Carregar o modelo
loaded_model = joblib.load(model_path)
print("Modelo carregado com sucesso!")

# Verificar se o modelo carregado produz o mesmo resultado
pred_loaded = loaded_model.predict(X_test)
print(f"Acurácia do modelo carregado (teste): {accuracy_score(y_test, pred_loaded):.4f}")

## 7) Rede Neural com TensorFlow/Keras

- Usaremos uma MLP (Perceptron Multicamadas) simples.
- Como o alvo é categórico com valores inteiros (0,1,2), usaremos a função de perda `sparse_categorical_crossentropy`.
- Incluiremos uma camada de normalização adaptada aos dados de treino.

In [ ]:
# Garantir tipos compatíveis para TensorFlow
X_train_tf = X_train.astype(np.float32)
X_test_tf = X_test.astype(np.float32)
y_train_tf = y_train.astype(np.int32)
y_test_tf = y_test.astype(np.int32)

# Camada de normalização baseada em estatísticas do treino
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(X_train_tf)

# Construção do modelo sequencial
model = keras.Sequential([
    normalizer,
    layers.Dense(16, activation="relu", kernel_initializer="he_normal"),
    layers.Dropout(0.2),
    layers.Dense(8, activation="relu", kernel_initializer="he_normal"),
    layers.Dense(3, activation="softmax") # 3 neurônios de saída para 3 classes
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.005),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

# Callback para parar o treino se a validação não melhorar
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=20, restore_best_weights=True
)

# Treinamento do modelo
history = model.fit(
    X_train_tf, y_train_tf,
    validation_split=0.2, # Usa 20% dos dados de treino para validação
    epochs=200,
    batch_size=16,
    callbacks=[early_stop],
    verbose=0 # Não mostra o log de cada época
)

print(f"Parâmetros treináveis: {model.count_params()}")
model.summary()

# Plotar curvas de treino
pd.DataFrame(history.history)[["loss", "val_loss", "accuracy", "val_accuracy"]].plot(figsize=(10, 6))
plt.title("Histórico de Treino (Perda e Acurácia)")
plt.xlabel("Épocas")
plt.grid(True)
plt.show()

# Avaliação final no conjunto de teste
test_loss, test_acc = model.evaluate(X_test_tf, y_test_tf, verbose=0)
print(f"\nAcurácia em teste (Keras MLP): {test_acc:.4f}\n")

# Relatório de classificação e matriz de confusão
y_pred_probs = model.predict(X_test_tf, verbose=0)
y_pred_nn = np.argmax(y_pred_probs, axis=1)
print("Relatório de classificação (Keras):\n", classification_report(y_test_tf, y_pred_nn, target_names=iris.target_names))

cm_nn = confusion_matrix(y_test_tf, y_pred_nn)
sns.heatmap(cm_nn, annot=True, fmt="d", cmap="Oranges",
            xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title("Matriz de confusão — Keras MLP")
plt.xlabel("Predito")
plt.ylabel("Verdadeiro")
plt.show()

## 8) Comparando Abordagens e Boas Práticas

- Em datasets pequenos e tabulares como o Iris, modelos do scikit-learn (ex.: SVC, Logistic Regression, RandomForest) costumam performar muito bem e treinar rapidamente. Nosso SVC com GridSearch atingiu 100% de acurácia no teste.
- Redes neurais são flexíveis e poderosas, mas podem ser um exagero para problemas simples, exigindo mais dados, regularização e ajuste fino.
- **Avaliação honesta**: Sempre mantenha um conjunto de teste separado e intocado. Use validação cruzada nos dados de treino para selecionar hiperparâmetros.
- **Reprodutibilidade**: Defina sementes (`random seeds`), registre versões de bibliotecas e salve o pipeline completo (pré-processamento + modelo).
- **Simplicidade primeiro**: Comece com um baseline simples e só então aumente a complexidade se necessário.

## 9) Próximos Passos

- Testar outros modelos (RandomForest, GradientBoosting, KNN).
- Explorar técnicas de explicabilidade (SHAP, Permutation Importance).
- Empacotar o pipeline para produção (versionamento, monitoramento de drift).

Parabéns! Você construiu um fluxo completo de ML com scikit-learn и TensorFlow usando o dataset Iris.